## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Camera Calibration

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        cv2.imshow('img',img)
        cv2.waitKey(100)

cv2.destroyAllWindows()

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

## Apply Undistortion

In [ ]:
def find_lane_pixels(binary_img, nwindows=9, margin=100, minpix=50):
    # create histogram of bottom half of the image
    histogram = np.sum(2*binary_img[binary_img.shape[0]//3:, :], axis=0)
    midpoint = histogram.shape[0]//2
    # initial guess for starting point of left and right lanes
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    window_height = binary_img.shape[0]//nwindows
    
    # identify x and y position of all nonzero pixels in image
    nonzero = binary_img.nonzero()
    nonzeroy = nonzero[0]
    nonzerox = nonzero[1]
    
    # current positions to be updated late for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base
    
    # create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    
    out_img = np.dstack((binary_img, binary_img, binary_img))
    for window in range(nwindows):
        # identify window boundaries in x and y
        win_y_low = binary_img.shape[0] - (window+1)*window_height
        win_y_high = binary_img.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(int(win_xleft_low),int(win_y_low)),(int(win_xleft_high),int(win_y_high)),(0,255,0), 2) 
        cv2.rectangle(out_img,(int(win_xright_low),int(win_y_low)),(int(win_xright_high),int(win_y_high)),(0,255,0), 2) 
        
        # identify nonzero pixels in x and y within window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

        # append indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # if found enought good indicies, recenter next window on their mean position 
        if len(good_left_inds) > minpix:
            leftx_current = np.mean(nonzerox[good_left_inds])
        if len(good_right_inds) > minpix:
            rightx_current = np.mean(nonzerox[good_right_inds])
            
    # concatenate the arrays of indices
 
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_img):
    # find lane pixels
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_img)
    
    # fit a second order polynomial
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # generate x and y values for plotting
    ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0])
    try:
        left_fitx = np.polyval(left_fit, ploty)
        right_fitx = np.polyval(right_fit, ploty)
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
        
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.imshow(out_img)

In [ ]:
def search_around(binary_img, margin=100):
    
    # grab activated pixels
    nonzero = binary_img.nonzero()
    nonzeroy = nonzero[0]
    nonzerox = nonzero[1]
    
    # set area of search based on activated x-values
    left_lane_inds = ((nonzerox > (np.polyval(left_fit, nonzeroy) - margin)) &
                      (nonzerox < (np.polyval(left_fit, nonzeroy) + margin)))
    right_lane_inds = ((nonzerox > (np.polyval(right_fit, nonzeroy) - margin)) & 
                       (nonzerox < (np.polyval(right_fit, nonzeroy) + margin)))
    
    # extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    # fit new polynomials
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # generate x and y for plotting
    ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0])
    left_fitx = np.polyval(left_fit, ploty)
    right_fitx = np.polyval(right_fit, ploty)
    
    ## visualization ##
    # create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_img, binary_img, binary_img))*255
    window_img = np.zeros_like(out_img)
    # color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # generate a polygon to illustrate the search window area
    # and recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # plot the polynomial lines onto the image
    plt.plot(left_fitx, ploty, color='yellow')
    plt.plot(right_fitx, ploty, color='yellow')
    plt.imshow(result)

In [92]:
class Line():
    def __init__(self):
        self._recent_fitted_xpts = []
        self._average_xpts = None
        self._average_fit = None
        self._current_fit = [np.array([False])]
        self._radius_of_curvature = None
        self._offset = None
        self._diffs = np.array([0, 0, 0], dtype='float')
        self._active_xpts = None
        self._active_ypts = None
        
class Lanes():
    def __init__(self, mtx, dist):
        self._left_lane = Line()
        self._right_lane = Line()
        self._detected = False
        self._mtx = mtx
        self._dist = dist
        self._warped_binary = None
        self._output_img = None
        self._nwindows = 9
        self._margin = 100
        self._min_active_pts = 50
    
    @staticmethod
    def display_img(img):
        cv2.imshow("Image", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    
    def lane_detection_pipline(self):
        pass

        
    def warp_binary_img(self, img, sobelx_thresh_min = 40, sobelx_thresh_max = 100, s_thresh_min=170, s_thresh_max = 255):
        # undistort image
        undistort = cv2.undistort(img, self._mtx, self._dist, None, self._mtx)
        # grayscale image
        gray = cv2.cvtColor(undistort, cv2.COLOR_BGR2GRAY)

        #sobel x
        sobelx = np.abs(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
        scaled_sobel = np.uint8(255*sobelx/np.max(sobelx))
        sxbinary = np.zeros_like(scaled_sobel)
        sxbinary[(scaled_sobel >= sobelx_thresh_min) & (scaled_sobel <= sobelx_thresh_max)] = 1

        # threshold color channel
        hls = cv2.cvtColor(undistort, cv2.COLOR_BGR2HLS)
        s_channel = hls[:, :, 2]
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

        # combining thresholded images
        combined_binary = np.zeros_like(sxbinary)
        combined_binary[(s_binary == 1) | (sxbinary == 1)] = 1
        # create binary image
        binary_image = combined_binary * 255
        # create perspective transform
        src = np.float32([[535, 500], [790, 500], [1105, 700], [280, 700]])
        dst = np.float32([[200, 0], [1000, 0], [1000, 680], [200, 680]])
        M = cv2.getPerspectiveTransform(src, dst)
        self._warped_binary = cv2.warpPerspective(binary_image, M, binary_image.shape[::-1], flags=cv2.INTER_LINEAR)
        self._output_img = np.dstack((self._warped_binary, self._warped_binary, self._warped_binary))
        self._ypts = np.linspace(0, self._warped_binary.shape[0]-1, self._warped_binary.shape[0])
    
    def find_lane_pixels(self, visualize = False):
        
        # create histogram of bottom half of the image
        histogram = np.sum(2*self._warped_binary[self._warped_binary.shape[0]//3:, :], axis=0)
        midpoint = histogram.shape[0]//2
        # initial guess for starting point of left and right lanes
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        window_height = self._warped_binary.shape[0]//self._nwindows

        # identify x and y position of all nonzero pixels in image
        nonzero = self._warped_binary.nonzero()
        nonzeroy = nonzero[0]
        nonzerox = nonzero[1]

        # current positions to be updated late for each window in nwindows
        leftx_current = leftx_base
        rightx_current = rightx_base

        # create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        for window in range(self._nwindows):
            # identify window boundaries in x and y
            win_y_low = self._warped_binary.shape[0] - (window+1)*window_height
            win_y_high = self._warped_binary.shape[0] - window*window_height
            win_xleft_low = leftx_current - self._margin
            win_xleft_high = leftx_current + self._margin
            win_xright_low = rightx_current - self._margin
            win_xright_high = rightx_current + self._margin

            if visualize:
                # Draw the windows on the visualization image
                cv2.rectangle(self._output_img,(int(win_xleft_low),int(win_y_low)),(int(win_xleft_high),int(win_y_high)),(0,255,0), 2) 
                cv2.rectangle(self._output_img,(int(win_xright_low),int(win_y_low)),(int(win_xright_high),int(win_y_high)),(0,255,0), 2) 

            # identify nonzero pixels in x and y within window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                              (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                               (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

            # append indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)

            # if found enought good indicies, recenter next window on their mean position 
            if len(good_left_inds) > self._min_active_pts:
                leftx_current = np.mean(nonzerox[good_left_inds])
            if len(good_right_inds) > self._min_active_pts:
                rightx_current = np.mean(nonzerox[good_right_inds])

        # concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
        self._left_lane._active_xpts = nonzerox[left_lane_inds]
        self._left_lane._active_ypts = nonzeroy[left_lane_inds]
        self._right_lane._active_xpts = nonzerox[right_lane_inds]
        self._right_lane._active_ypts = nonzeroy[right_lane_inds]
        
        # fit a second order polynomial
        self._left_lane._current_fit = np.polyfit(self._left_lane._active_ypts, self._left_lane._active_xpts, 2)
        self._right_lane._current_fit = np.polyfit(self._right_lane._active_ypts, self._right_lane._active_xpts, 2)
        try:
            left_fitx = np.polyval(self._left_lane._current_fit, self._ypts)
            right_fitx = np.polyval(self._right_lane._current_fit, self._ypts)
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            left_fitx = 1*ploty**2 + 1*ploty
            right_fitx = 1*ploty**2 + 1*ploty
        
        self._left_lane._recent_fitted_xpts.append(left_fitx)
        self._right_lane._recent_fitted_xpts.append(right_fitx)
        self._detected = True
        
        if visualize:
            plt.plot(left_fitx, self._ypts, color='yellow')
            plt.plot(right_fitx, self._ypts, color='yellow')
            self._output_img[self._left_lane._active_ypts, self._left_lane._active_xpts] = [255, 0, 0]
            self._output_img[self._right_lane._active_ypts, self._right_lane._active_xpts] = [0, 200, 255 ]
            plt.imshow(self._output_img)
            
    def search_around(self, visualize=False):
    
        # grab activated pixels
        nonzero = self._warped_binary.nonzero()
        nonzeroy = nonzero[0]
        nonzerox = nonzero[1]

        # set area of search based on activated x-values
        left_lane_inds = ((nonzerox > (np.polyval(self._left_lane._current_fit, nonzeroy) - self._margin)) &
                          (nonzerox < (np.polyval(self._left_lane._current_fit, nonzeroy) + self._margin)))
        right_lane_inds = ((nonzerox > (np.polyval(self._right_lane._current_fit, nonzeroy) - self._margin)) & 
                           (nonzerox < (np.polyval(self._right_lane._current_fit, nonzeroy) + self._margin)))
        if ((left_lane_inds.size < self._min_active_pts * self._nwindows) and (right_lane_inds.size < self._min_active_pts * self._nwindows)):
            self._detected = False
        else:
        
            # extract left and right line pixel positions
            self._left_lane._active_xpts = nonzerox[left_lane_inds]
            self._left_lane._active_ypts = nonzeroy[left_lane_inds]
            self._right_lane._active_xpts = nonzerox[right_lane_inds]
            self._right_lane._active_ypts = nonzeroy[right_lane_inds]

            # fit new polynomial
            self._left_lane._current_fit = np.polyfit(self._left_lane._active_ypts, 
                                                      self._left_lane._active_xpts, 2)
            self._right_lane._current_fit = np.polyfit(self._right_lane._active_ypts, 
                                                       self._right_lane._active_xpts, 2)
            
            # generate x for plotting
            left_fitx = np.polyval(self._left_lane._current_fit, self._ypts)
            right_fitx = np.polyval(self._right_lane._current_fit, self._ypts)
            self._left_lane._recent_fitted_xpts.append(left_fitx)
            self._right_lane._recent_fitted_xpts.append(right_fitx)
        
        if visualize:
            window_img = np.zeros_like(self._output_img)
            # color in left and right line pixels
            self._output_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
            self._output_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 200, 255 ]

            # generate a polygon to illustrate the search window area
            # and recast the x and y points into usable format for cv2.fillPoly()
            left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-self._margin, self._ypts]))])
            left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+self._margin, self._ypts])))])
            left_line_pts = np.hstack((left_line_window1, left_line_window2))
            right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-self._margin, self._ypts]))])
            right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+self._margin, self._ypts])))])
            right_line_pts = np.hstack((right_line_window1, right_line_window2))

            # draw the lane onto the warped blank image
            cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
            cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
            self._output_img = cv2.addWeighted(self._output_img, 1, window_img, 0.3, 0)

            # plot the polynomial lines onto the image
            plt.plot(left_fitx, self._ypts, color='yellow')
            plt.plot(right_fitx, self._ypts, color='yellow')
            plt.imshow(self._output_img)

In [94]:
lanes = Lanes(mtx, dist)
test_img = cv2.imread("./test_images/test5.jpg")
lanes.warp_binary_img(test_img)
lanes.find_lane_pixels(False)
lanes.search_around(False)
# fit_polynomial(warped_binary)
# search_around(warped_binary)